In [ ]:
import torch
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as trans
import torch.nn.functional as F
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import pickle
import random

IMAGENET_TRANSFORM = trans.Compose([trans.ToTensor()])

In [ ]:
# proj = '/content/drive/MyDrive/SMAI_proj/'
# testset = dset.ImageFolder(proj+'Imagenet16v2/val', IMAGENET_TRANSFORM)
testset = dset.ImageFolder('/content/drive/MyDrive/SMAI_proj/subset/val', IMAGENET_TRANSFORM)

In [ ]:
# idx = torch.randperm(len(testset))
# pickle.dump(idx,open('/content/drive/MyDrive/SMAI_proj/idx','wb'))
idx = pickle.load(open('idx','rb'))

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(idx)

3925

In [ ]:
class Simba:
  def __init__(self, model_name, epsilon = 0.1):
    self.model = getattr(models, model_name)(pretrained=True).cuda()
    self.epsilon = epsilon
    self.model.eval()
  
  def getAttackIteration(self,img, itr = 10000): # max iteration: 12288
    data,_ = img
    res = self.model(data.unsqueeze(0).cuda()).cpu()
    res = torch.squeeze(res)
    # print(res.shape)
    actual_class = torch.argmax(res)
    last_prob = F.softmax(res,dim=-1)[actual_class]
    length = torch.numel(data)
    rand = torch.randperm(length)
    image = data
    for i in tqdm(range(itr)):
      noise = torch.zeros(length)
      noise[rand[i]] += self.epsilon 
      # check addition
      new_image = torch.flatten(image) + noise
      new_image = new_image.reshape(data.shape)
      res = self.model(new_image.unsqueeze(0).cuda()).cpu()
      res = torch.squeeze(res)
      new_class = torch.argmax(res)
      if new_class != actual_class:
        image = new_image
        break
      new_prob = F.softmax(res,dim=-1)[actual_class]
      if new_prob > last_prob:
        #check substraction
        new_image = torch.flatten(image) - noise
        new_image = new_image.reshape(data.shape)
        res = self.model(new_image.unsqueeze(0).cuda()).cpu()
        res = torch.squeeze(res)
        new_class = torch.argmax(res)
        if new_class != actual_class:
          image = new_image
          break
        new_prob = F.softmax(res,dim=-1)[actual_class]
        if new_prob > last_prob:
          image = new_image
          last_prob = new_prob
      else:
        image = new_image
        last_prob = new_prob
      # print(last_prob)
    return new_image, i


In [ ]:
labels = pickle.load(open('labels.pkl','rb'))
len(testset)

3925

In [ ]:
attacker = Simba('mobilenet_v2',0.3)
arr = []
img_data = []
for i in range(100):
  print('attack on image',i)
  img, k = attacker.getAttackIteration(testset[idx[i]])
  x,y = testset[idx[i]]
  if k != 9999:
    arr.append(k)
    img_data.append([x,img])

    fig,a =  plt.subplots(1,2)

    res = attacker.model(x.unsqueeze(0).cuda()).cpu()
    res = torch.squeeze(res)
    actual_class = int(torch.argmax(res))
    a[0].set_title(str(labels[actual_class]))
    a[0].imshow(np.transpose(x,(1,2,0)))

    res = attacker.model(img.unsqueeze(0).cuda()).cpu()
    res = torch.squeeze(res)
    actual_class = int(torch.argmax(res))
    a[1].set_title(str(labels[actual_class]))
    a[1].imshow(np.transpose(img,(1,2,0)))
    plt.tight_layout()
    plt.show()
    print('successful attack ratio:',len(arr)/(i+1))
    print(np.average(arr)) 
    pickle.dump(img_data,open('img_udata_03_100','wb'))
    pickle.dump(arr,open('uarr_03_100','wb'))
    pickle.dump(i,open('i_03_100u','wb'))

In [ ]:
# print(len(img_data))
# print(len(arr))
# print(i)

In [ ]:
# pickle.dump(img_data,open('/content/drive/MyDrive/SMAI_proj/saved/mobilenet_v2/img_data','wb'))
# pickle.dump(arr,open('/content/drive/MyDrive/SMAI_proj/saved/mobilenet_v2/arr','wb'))
# pickle.dump(i,open('/content/drive/MyDrive/SMAI_proj/saved/mobilenet_v2/i','wb'))